In [2]:
import statsmodels.api as sm
import statsmodels.stats.sandwich_covariance as sw
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv(OK.csv)


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [44]:
df_tol['Date']

0      2021-01-04
1      2021-01-05
2      2021-01-06
3      2021-01-07
4      2021-01-08
          ...    
748    2023-12-22
749    2023-12-26
750    2023-12-27
751    2023-12-28
752    2023-12-29
Name: Date, Length: 753, dtype: object

In [45]:
df_tol['Close']

0       42.660000
1       42.410000
2       42.360001
3       44.220001
4       42.970001
          ...    
748    103.570000
749    103.470001
750    104.059998
751    103.529999
752    102.790001
Name: Close, Length: 753, dtype: float64

In [46]:
#--y = np.asarray(df_tol['Close']).astype(float)
X = pd.to_datetime(df_tol['Date']).apply(lambda x: x.toordinal()) 
X = np.asarray(sm.add_constant(X))

In [47]:
model = sm.OLS(df_tol['Close'], X).fit()

In [48]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Close   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.229
Method:                 Least Squares   F-statistic:                     224.5
Date:                Thu, 06 Jun 2024   Prob (F-statistic):           1.37e-44
Time:                        13:56:34   Log-Likelihood:                -2882.1
No. Observations:                 753   AIC:                             5768.
Df Residuals:                     751   BIC:                             5777.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.417e+04    949.775    -14.919      0.0

In [49]:
hac_cov = sw.cov_hac(model, 1)

In [50]:
s = model.get_robustcov_results(cov_type='HAC', maxlags=1)
print(s.summary())

                            OLS Regression Results                            
Dep. Variable:                  Close   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.229
Method:                 Least Squares   F-statistic:                     110.7
Date:                Thu, 06 Jun 2024   Prob (F-statistic):           3.04e-24
Time:                        13:56:34   Log-Likelihood:                -2882.1
No. Observations:                 753   AIC:                             5768.
Df Residuals:                     751   BIC:                             5777.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.417e+04   1352.214    -10.479      0.0